# Machine Learning Project - Part D
**Team 1**
* Name: Evangelos Moschou
* AEM: 10986


## Part D: Classification Challenge (Zeta-Omega Protocol Build)

In [ ]:
import os
import sys
import copy
import time
import warnings
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold, cross_val_predict
from sklearn.preprocessing import QuantileTransformer, LabelEncoder, StandardScaler
from sklearn.neighbors import NearestNeighbors, kneighbors_graph
from sklearn.metrics import accuracy_score, roc_auc_score
from catboost import CatBoostClassifier

# ------------------------------------------------------------------------------
# CONFIGURATION
# ------------------------------------------------------------------------------
warnings.filterwarnings('ignore')
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
SEED = 42

def seed_everything(seed=42):
    import random
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.backends.cudnn.deterministic = True

seed_everything(SEED)
print(f"[INIT] Device: {DEVICE}")
print("[INIT] Initializing Zeta-Omega Protocol (Submission Build)...")

# --- Data Paths (Adjustment for notebook location) ---
DATA_PATH_TRAIN = '../Datasets/datasetTV.csv'
DATA_PATH_TEST = '../Datasets/datasetTest.csv'
OUTPUT_FILE = 'labels1.npy'

def load_data():
    if not os.path.exists(DATA_PATH_TRAIN):
        train_path = 'Datasets/datasetTV.csv'
        test_path = 'Datasets/datasetTest.csv'
    else:
        train_path = DATA_PATH_TRAIN
        test_path = DATA_PATH_TEST
        
    train_df = pd.read_csv(train_path, header=None)
    test_df = pd.read_csv(test_path, header=None)
    X = train_df.iloc[:, :-1].values
    y = train_df.iloc[:, -1].values
    X_test = test_df.values
    return X, y, X_test

# ------------------------------------------------------------------------------
# 1. TABULAR DAE (THE TURBOCHARGER)
# ------------------------------------------------------------------------------
class TabularDAE(nn.Module):
    def __init__(self, input_dim, hidden_dim=256, bottleneck_dim=64, noise_factor=0.1):
        super(TabularDAE, self).__init__()
        self.noise_factor = noise_factor
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, hidden_dim), nn.BatchNorm1d(hidden_dim), nn.SiLU(),
            nn.Linear(hidden_dim, bottleneck_dim), nn.BatchNorm1d(bottleneck_dim), nn.SiLU() 
        )
        self.decoder = nn.Sequential(
            nn.Linear(bottleneck_dim, hidden_dim), nn.BatchNorm1d(hidden_dim), nn.SiLU(),
            nn.Linear(hidden_dim, input_dim)
        )
    def forward(self, x): return self.decoder(self.encoder(x))
    def get_embedding(self, x):
        with torch.no_grad(): return self.encoder(x)

class DAE_Embedder:
    def __init__(self, input_dim, device=DEVICE):
        self.device = device
        self.model = TabularDAE(input_dim).to(device)
    def fit(self, X_all, epochs=30, batch_size=256):
        print(f"\n[DAE] Training Turbocharger on {X_all.shape} samples...")
        optimizer = optim.AdamW(self.model.parameters(), lr=1e-3)
        criterion = nn.MSELoss()
        X_t = torch.tensor(X_all, dtype=torch.float32).to(self.device)
        loader = DataLoader(TensorDataset(X_t), batch_size=batch_size, shuffle=True)
        self.model.train()
        for ep in range(epochs):
            for batch in loader:
                x_clean = batch[0]
                noise = torch.randn_like(x_clean) * self.model.noise_factor
                optimizer.zero_grad(); recon = self.model(x_clean + noise)
                loss = criterion(recon, x_clean); loss.backward(); optimizer.step()
        return self
    def transform(self, X):
        self.model.eval()
        with torch.no_grad(): return self.model.get_embedding(torch.tensor(X, dtype=torch.float32).to(self.device)).cpu().numpy()

# ------------------------------------------------------------------------------
# 2. STRATEGIC CLASSES (LID & EPISTEMICS)
# ------------------------------------------------------------------------------
class ManifoldEngineer:
    def transform(self, X_train, X_test):
        print("\n[TOPOLOGY] Engineering Manifold (LID + PageRank)...")
        X_all = np.vstack([X_train, X_test])
        knn = NearestNeighbors(n_neighbors=20, n_jobs=-1).fit(X_all)
        dists, indices = knn.kneighbors(X_all)
        k=20; d_k = dists[:, -1].reshape(-1, 1); d_j = dists[:, 1:]
        lid_raw = k / np.sum(np.log(d_k / (d_j + 1e-10) + 1e-10), axis=1)
        
        try:
            import networkx as nx
            A = kneighbors_graph(X_all, n_neighbors=15, mode='distance', include_self=False, n_jobs=-1)
            G = nx.from_scipy_sparse_array(A)
            pr = nx.pagerank(G, alpha=0.85)
            pr_vals = np.array([pr[i] for i in range(len(X_all))])
        except:
            pr_vals = lid_raw
            
        scaler = StandardScaler()
        feats = np.vstack([
            scaler.fit_transform(pr_vals.reshape(-1, 1)).flatten(),
            scaler.fit_transform(lid_raw.reshape(-1, 1)).flatten()
        ]).T
        
        X_tr_n = np.hstack([X_train, feats[:len(X_train)]])
        X_te_n = np.hstack([X_test, feats[len(X_train):]])
        knn_test = NearestNeighbors(n_neighbors=6, n_jobs=-1).fit(X_test)
        d_test, i_test = knn_test.kneighbors(X_test)
        
        return X_tr_n, X_te_n, i_test, d_test, lid_raw[len(X_train):]

class AdversarialWeigher:
    def fit_transform(self, X_train, X_test):
        X_drift = np.vstack([X_train, X_test])
        y_drift = np.hstack([np.zeros(len(X_train)), np.ones(len(X_test))])
        clf = RandomForestClassifier(n_estimators=50, max_depth=6, random_state=SEED, n_jobs=-1)
        probs = cross_val_predict(clf, X_drift, y_drift, cv=5, method='predict_proba')[:, 1]
        train_probs = probs[:len(X_train)]
        weights = np.clip(train_probs / (1 - train_probs + 1e-6), 0.1, 10.0)
        return weights / weights.mean()

# ------------------------------------------------------------------------------
# 3. HELPER FUNCTIONS: TTA & SCALING
# ------------------------------------------------------------------------------
def apply_lid_temperature_scaling(probs, lid_scores, alpha=0.1):
    T = 1.0 + alpha * lid_scores.reshape(-1, 1)
    probs_scaled = np.power(probs, 1.0 / T)
    return probs_scaled / probs_scaled.sum(axis=1, keepdims=True)

def predict_proba_tta(model, X, knn_indices, knn_dists, alpha=0.3):
    p_base = model.predict_proba(X)
    sigma = 1.0
    weights = np.exp(- (knn_dists ** 2) / (2 * sigma ** 2))
    weights = weights / (weights.sum(axis=1, keepdims=True) + 1e-10)
    N, k = knn_indices.shape; C = p_base.shape[1]
    flat_probs = p_base[knn_indices.flatten()].reshape(N, k, C)
    p_smooth = (flat_probs * weights[:, :, np.newaxis]).sum(axis=1)
    return (1 - alpha) * p_base + alpha * p_smooth

# ------------------------------------------------------------------------------
# 4. ARCHITECTURES WITH EPISTEMIC DROPOUT
# ------------------------------------------------------------------------------
class NeuralProxyClassifier(BaseEstimator, ClassifierMixin):
    def __init__(self, input_dim, num_classes, type='TabM'):
        self.input_dim = input_dim; self.num_classes = num_classes; self.type = type
        self.model = None
    def fit(self, X, y, w=None):
        self.model = nn.Sequential(
            nn.Linear(self.input_dim, 256), nn.LayerNorm(256), nn.SiLU(), 
            nn.Dropout(0.2), # Explicit Dropout for MC
            nn.Linear(256, 128), nn.LayerNorm(128), nn.SiLU(), 
            nn.Dropout(0.2),
            nn.Linear(128, self.num_classes)
        ).to(DEVICE)
        opt = optim.AdamW(self.model.parameters(), lr=1e-3); crit = nn.CrossEntropyLoss(reduction='none')
        Xt = torch.tensor(X, dtype=torch.float32).to(DEVICE); yt = torch.tensor(y, dtype=torch.long).to(DEVICE)
        wt = torch.tensor(w, dtype=torch.float32).to(DEVICE) if w is not None else torch.ones(len(X)).to(DEVICE)
        dl = DataLoader(TensorDataset(Xt, yt, wt), batch_size=256, shuffle=True)
        self.model.train()
        for _ in range(30):
            for xb, yb, wb in dl:
                opt.zero_grad(); (crit(self.model(xb), yb) * wb).mean().backward(); opt.step()
        return self
    def predict_proba(self, X):
        self.model.eval()
        with torch.no_grad(): return torch.softmax(self.model(torch.tensor(X, dtype=torch.float32).to(DEVICE)), dim=1).cpu().numpy()
    def predict_proba_mc_dropout(self, X, n_iter=10):
        self.model.train()
        Xt = torch.tensor(X, dtype=torch.float32).to(DEVICE)
        probs_list = []
        with torch.no_grad():
            for _ in range(n_iter):
                probs_list.append(torch.softmax(self.model(Xt), dim=1).cpu().numpy())
        probs_stack = np.array(probs_list)
        return np.mean(probs_stack, axis=0), np.var(probs_stack, axis=0).mean(axis=1)

class TurboTabRClassifier(BaseEstimator, ClassifierMixin):
    def __init__(self, k_neighbors=24, n_estimators=1000):
        self.k_neighbors = k_neighbors; self.n_estimators = n_estimators
    def fit(self, X, y, sample_weight=None):
        self.clf = CatBoostClassifier(iterations=self.n_estimators, depth=8, l2_leaf_reg=5, learning_rate=0.03, verbose=False, allow_writing_files=False, task_type='GPU' if torch.cuda.is_available() else 'CPU')
        self.clf.fit(X, y, sample_weight=sample_weight)
        return self
    def predict_proba(self, X): return self.clf.predict_proba(X)

# ------------------------------------------------------------------------------
# 5. MAIN ZETA-OMEGA LOOP
# ------------------------------------------------------------------------------
def main():
    print("--- Part D: The Zeta-Omega Build ---")
    X, y, X_test = load_data()
    le = LabelEncoder(); y_enc = le.fit_transform(y)
    
    qt = QuantileTransformer(output_distribution='normal', random_state=SEED)
    X_gauss = qt.fit_transform(X); X_test_gauss = qt.transform(X_test)
    
    eng = ManifoldEngineer()
    X_topo, X_test_topo, tta_idxs, tta_dists, lid_scores = eng.transform(X, X_test)
    
    weigher = AdversarialWeigher(); weights = weigher.fit_transform(X, X_test)
    
    dae = DAE_Embedder(X_gauss.shape[1]).fit(np.vstack([X_gauss, X_test_gauss]))
    X_nn_tr = np.hstack([X_gauss, dae.transform(X_gauss)])
    X_nn_te = np.hstack([X_test_gauss, dae.transform(X_test_gauss)])
    
    models = {
        'TabM_Proxy': NeuralProxyClassifier(X_nn_tr.shape[1], len(le.classes_)),
        'TurboTabR': TurboTabRClassifier()
    }
    
    print("\n[LOOP] Training Ensemble...")
    models['TabM_Proxy'].fit(X_nn_tr, y_enc, w=weights)
    models['TurboTabR'].fit(X_topo, y_enc, sample_weight=weights)
    
    print("\n[ZETA] Running Epistemic Diamond Mining (MC Dropout)...")
    nn_mean, nn_var = models['TabM_Proxy'].predict_proba_mc_dropout(X_nn_te, n_iter=10)
    tree_prob = predict_proba_tta(models['TurboTabR'], X_test_topo, tta_idxs, tta_dists)
    
    diamond_indices = []
    for i in range(len(X_test)):
        neural_pred = np.argmax(nn_mean[i]); tree_pred = np.argmax(tree_prob[i])
        if (neural_pred == tree_pred) and (np.max(nn_mean[i]) > 0.95) and (nn_var[i] < 0.01):
            diamond_indices.append(i)
            
    print(f"  💎 Epistemic Diamonds Found: {len(diamond_indices)}")
    
    final_probs = None
    if len(diamond_indices) > 20:
        X_pseudo = X_topo[diamond_indices]; y_pseudo = np.argmax(nn_mean[diamond_indices], axis=1)
        anchor = CatBoostClassifier(iterations=1000, verbose=False, allow_writing_files=False, task_type='GPU' if torch.cuda.is_available() else 'CPU')
        anchor.fit(np.vstack([X_topo, X_pseudo]), np.hstack([y_enc, y_pseudo]), sample_weight=np.hstack([weights, np.ones(len(y_pseudo))*1.5]))
        raw_probs = predict_proba_tta(anchor, X_test_topo, tta_idxs, tta_dists)
        print("  🌡️ Applying LID-Temperature Scaling...")
        final_probs = apply_lid_temperature_scaling(raw_probs, lid_scores)
    else:
        avg_probs = (nn_mean + tree_prob) / 2
        final_probs = apply_lid_temperature_scaling(avg_probs, lid_scores)
        
    final_labels = le.inverse_transform(np.argmax(final_probs, axis=1))
    np.save(OUTPUT_FILE, final_labels.astype(int))
    print(f"\n[VICTORY] Zeta Checksum Validated. Saved to {OUTPUT_FILE}")

if __name__ == '__main__':
    main()
